# Libraries

In [ ]:
# import additional stuff needed later on:
import json
import random
import time


In [ ]:
# import Paillier library
!pip install phe
import phe
from phe import paillier

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Ruf

In [ ]:
pubkey, privkey = paillier.generate_paillier_keypair(n_length=1024)

In [ ]:
print("Public Key:")
print(pubkey.n)
print(len(str(pubkey.n)))
print(pubkey.g)
print(pubkey.g - pubkey.n)
if pubkey.n%2:
    print("N is odd.")
print()

print("Private Key:")
print(privkey.p)
print(len(str(privkey.p)))
print(privkey.q)
print(len(str(privkey.q)))

Public Key:
97755977927658485539115917631349267412813866748490148895487169771189654243158720903897972004194656407248673003290186642858509024608419168822915681980373263778962636217115921785483975767572897948718286639986348029027973392214589054770104008716948493169447838599654315029779777954008942126893455472295159350261
308
97755977927658485539115917631349267412813866748490148895487169771189654243158720903897972004194656407248673003290186642858509024608419168822915681980373263778962636217115921785483975767572897948718286639986348029027973392214589054770104008716948493169447838599654315029779777954008942126893455472295159350262
1
N is odd.

Private Key:
8701335285220836168582003633477496866818608030615179830246990401065440625052862546685645319297137365650353589473722887175061052621716289618896079899231049
154
11234595004481255387316770801865342684167200137378796043883779494874289464809063087853540691498200341581189644223690713267389647915424105405861940364218189
155


In [ ]:
# p = privkey.p
# q = privkey.q

# g = pubkey.g
# n = pubkey.n

In [ ]:
m1 = 3.14
m2 = 10.02

M1 = pubkey.encrypt(m1)
M2 = pubkey.encrypt(m2)

In [ ]:
M3 = M1 + M2

# Create some constants
n1 = 2
n2 = 0.125

M4 = M1 * n1

M5 = M1 + n2

# Multiplication of two ciphertexts are not supported
# M6 = M1 * M2

In [ ]:
print(M3.ciphertext())
print(M4.ciphertext())
print(M5.ciphertext())

2361723974015106489756358439574999910063475430601857155660521712118788295462843931648403105364153170888025791508787615604328766447776955547789912934107279780135496613821778612176386240590030345444198650187072232579603220460165517347668073367011117917466633896382991547841123194351713978094027731348789440711487994992944714578078176511639499307778337869610861525675717310070750576700470147668689841783132190077024870548619962927751516735178728648238030224684190643993517942295448864214309042796186177702597136045806883027398185513374786582033435288360414070984848534633568148237725016678190508667414551368694515867921
75279587862975193714380326170814574779680725653926993431228979434409957100221852268073057772568293061807423866508076712423532769549892894461113998967174800201963472682598245990632005193766785236672477347021970003097233037254247835562546946904615122323840942123718552473450359363294380628261327342890202214649168701558953232211762321631353008101027441927669817750324510800380827485296

In [ ]:
m3 = privkey.decrypt(M3)
m4 = privkey.decrypt(M4)
m5 = privkey.decrypt(M5)

print(m3)
print(m4)
print(m5)

print("Original values:")
print(m1 + m2)
print(n1 * m1)
print(m1 + n2)

13.16
6.28
3.265
Original values:
13.16
6.28
3.265


# SBD

In [ ]:
def Encrypted_LSB(T):
    # Bob:
    # print("E1")
    r = random.randint(1, privkey.q)
    E_r = pubkey.encrypt(r)
    # print("E12")
    Y = T + E_r

    # Alice:
    # print("E2")
    y = privkey.decrypt(Y)
    temp = 0
    if y%2:                 # y is odd
        temp = pubkey.encrypt(1)
    else:                   # y is even
        temp = pubkey.encrypt(0)

    # Bob:
    # print("E3")
    ans = 0
    if r%2:
        ans = pubkey.encrypt(1) - temp
    else:
        ans = temp
    return ans



def SVR(E_x, l1):
    # Bob:
    sum = 0
    t = 1
    for i in range(len(l1)):
        sum = sum + l1[i] * t
        t *= 2
    V = sum - E_x
    rr = random.randint(1, 1000)
    W = V * rr

    # Alice:
    D_W = privkey.decrypt(W)
    if D_W:
        return 0
    else:
        return 1

In [ ]:
def SBD(E_x, m):

    # Stage 1:

    # l = n/2 + 1
    T = E_x
    l1 = []
    # print(1)
    for i in range(m):
        l1.append(Encrypted_LSB(T))
        # print(2)

        Z = T - l1[len(l1) - 1]
        T = Z / 2
        # print(3)
    # print(4)
    # Stage 2:

    gamma = SVR(E_x, l1)
    if gamma:
        return l1
    else:
        return SBD(E_x, m)

In [ ]:
def CountSBD(E_x, m, count):

    # Stage 1:

    # l = n/2 + 1
    T = E_x
    l1 = []
    # print(1)
    for i in range(m):
        l1.append(Encrypted_LSB(T))
        # print(2)

        Z = T - l1[len(l1) - 1]
        T = Z / 2
        # print(3)
    # print(4)
    # Stage 2:

    gamma = SVR(E_x, l1)
    if gamma:
        return count + 1, l1
    else:
        return CountSBD(E_x, m, count + 1)

In [ ]:
x = 109

E_x = pubkey.encrypt(x)
m = x.bit_length()
count = 0
count, l1 = CountSBD(E_x, m, 0)
print("OUTPUT:")
print(count)
for i in range(len(l1) - 1, -1, -1):
    print(privkey.decrypt(l1[i]), end = "")

OUTPUT:
12
1101101

In [ ]:
values_count = 20
l = random.sample(range(1, 201), values_count)
countl = []
totalC = 0

for x in l:
    E_x = pubkey.encrypt(x)
    m = x.bit_length()
    count = 0
    count, l1 = CountSBD(E_x, m, 0)
    countl.append([count, x])
    totalC += count

print("SBD runs for = ", totalC, "times.")
print("Average number of SBD calls = ", totalC/values_count)

SBD runs for =  1670 times.
Average number of SBD calls =  83.5
